In [1]:
import numpy as np  
import scipy as sc
import pandas as pd
import matplotlib as plt
import seaborn as sns
import os
%matplotlib inline

In [3]:
# ALL INDICES READ INTO DF

from collections import OrderedDict
series = OrderedDict()
for filename in os.listdir("datasets/indices/"):
    
    path = "datasets/indices/" + filename
    cols = ['Date', 'Close', 'Volume']
    col1 = str(filename.lower().split(".")[0] + '_close')
    col2 = str(filename.lower().split(".")[0] + '_volume')
    
    try:
        series[filename] = pd.read_csv(path, parse_dates=True, header=0, index_col=0, usecols=cols)
        series[filename] = series[filename].rename(columns={'Date':'date','Close':col1,'Volume':col2})
    except:
        pass
    #    if filename == "S&P500-Energy.csv":
    #        series[filename] = pd.read_csv(path, parse_dates=True, header=0, index_col=0, usecols=['Date','Close'])
    #        series[filename] = series[filename].rename(columns={'Date':'date','Close':col1})

df_indices = pd.DataFrame()
for each in series.keys():
    df_indices = pd.concat([df_indices, series[each]], axis=1).dropna()
df_indices.head()

,s&p500_close,s&p500_volume
Date,,
2007-08-31,1473.989990,2731610000
2007-09-04,1489.420044,2766600000
2007-09-05,1472.290039,2991600000
2007-09-06,1478.550049,2459590000
2007-09-07,1453.550049,3191080000


In [ ]:
# ALL SENTIMENT DATA READ INTO DF

series = OrderedDict()
for filename in os.listdir("datasets/sentiment/"):
    
    path = "datasets/sentiment/" + filename
    cols = ['Date', 'Sentiment', 'News Volume', 'News Buzz']
    col1 = str(filename[4:].lower().split("_")[0] + '_sentiment')
    col2 = str(filename[4:].lower().split("_")[0] + '_newsvol')
    col3 = str(filename[4:].lower().split("_")[0] + '_buzz')
    
    try:
        series[filename] = pd.read_csv(path, parse_dates=True, header=0, index_col=0, usecols=cols)
        series[filename] = series[filename].rename(columns={'Date':'date','Sentiment':col1,'News Volume':col2, 'News Buzz':col3})
    except:
        pass

df_sentiment = pd.DataFrame()
for each in series.keys():
    df_sentiment = pd.concat([df_sentiment, series[each]], axis=1).dropna()
#df_sentiment.head()

In [ ]:
# ALL STOCKS READ INTO DF

series = OrderedDict()
for filename in os.listdir("datasets/stock prices/"):
    
    path = "datasets/stock prices/" + filename
    cols = ['Date', 'Close', 'Volume']
    col1 = str(filename.lower().split(".")[0] + '_close')
    col2 = str(filename.lower().split(".")[0] + '_volume')
    
    try:
        series[filename] = pd.read_csv(path, parse_dates=True, header=0, index_col=0, usecols=cols)
        series[filename] = series[filename].rename(columns={'Date':'date','Close':col1,'Volume':col2})
    except:
        pass

df_stocks = pd.DataFrame()
for each in series.keys():
    df_stocks = pd.concat([df_stocks, series[each]], axis=1).dropna()
#df_stocks.head()

In [ ]:
# CONVERTING TO LOG RETURNS

df_pre = pd.concat([df_stocks, df_indices],axis=1).dropna()

# Stashing column names
all_cols = df_pre.columns
prices = [k for k in all_cols if 'close' in k]
mat = df_pre[prices].as_matrix()
matnew = np.zeros((len(mat)-1,len(mat[0])))
for i in xrange(len(matnew)):
    matnew[i] = np.divide(np.subtract(mat[i+1], mat[i]), mat[i])
matnew1 = np.cumsum(np.log(1+ matnew), axis=0)

df_pre = df_pre.iloc[1:,]
df_pre = df_pre.drop(prices, axis=1)
df_pre = pd.concat([df_pre, pd.DataFrame(matnew1, columns=prices, index=df_pre.index)], axis=1)
df_pre = df_pre.reindex_axis(sorted(df_pre.columns), axis=1)

#print df_pre.info()

#VOLATILITY NORMALIZING

x = df_pre.abb_close
y1 = np.zeros(len(x))

#Detrending 1 - SLOW MOVING AVERAGE - PICK WINDOW LENGTH
win = 60
for i in range(len(x)-60):
    y1[i] = np.mean(x[i:i+60])
y1 = pd.Series(y1, index=x.index)
x.plot(color='yellow')
y.plot(color='blue')
z1 = pd.Series.subtract(x,y1)
z1.plot()

#Detrending 2 - SCIPY DETREND
from scipy import signal
y2 = pd.Series(signal.detrend(x), index=x.index)
y2.plot(color='green')

#Detrending 3 - LINEAR REGRESSION
from sklearn.linear_model import LinearRegression

mod = LinearRegression()
r = np.arange(len(x)).reshape((len(x),1))
v = x.as_matrix().reshape((len(x),1))
mod.fit(r,v)
trend = mod.predict(v)
y3 = pd.Series(trend.reshape(len(trend),),index=x.index)
y3.plot()

In [ ]:
# FINAL MERGE

df = pd.concat([df_pre, df_sentiment],axis=1).dropna()
print len(df)
print df.info()
print df.head()